# ARIMA MODEL 

In [1]:
import pmdarima as pm
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit
import pandas as pd
import numpy as np
import datetime
import requests
import warnings

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA
from sklearn.preprocessing import StandardScaler
# from fbprophet import Prophet
# from fbprophet.plot import plot_plotly, add_changepoints_to_plot
# from keras.models import Sequential
# from keras.layers import Dense

# from keras.optimizers import Adam

warnings.filterwarnings('ignore')

/Users/amandatsuetaki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/amandatsuetaki/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


## Preprocessing 

In [6]:
czech  = pd.read_csv('univariate_models/czechia_diff.csv')
eston  = pd.read_csv('univariate_models/estonia_diff.csv')
franc  = pd.read_csv('univariate_models/france_diff.csv')
malay  = pd.read_csv('univariate_models/malaysia_diff.csv')
belgi  = pd.read_csv('univariate_models/univariate_df_Belgium.csv')
chile  = pd.read_csv('univariate_models/univariate_df_Chile.csv')
irela  = pd.read_csv('univariate_models/univariate_df_Ireland.csv')
israe  = pd.read_csv('univariate_models/univariate_df_Israel.csv')
italy  = pd.read_csv('univariate_models/univariate_df_Italy.csv')
us  = pd.read_csv('univariate_models/us_diff.csv')


In [18]:
# setting index to the date time value 
def index_dt(c):
    c.set_index('date')

for x in [czech, eston, franc, malay,belgi, chile, irela, israe, italy, us]:
    x = index_dt(x)   

## Data Spliting 
In time series prediction, it is crucial to handle the temporal aspect properly when splitting the data into training and testing sets. Unlike random data, time series data has a sequential structure, and random splitting can lead to information leakage and overly optimistic performance estimates.

In [19]:
#sort index
for x in [czech, eston, franc, malay,belgi, chile, irela, israe, italy, us]:
    x = x.sort_index()

In [20]:
train_size = int(len(czech) * 0.8)
czech_train, czech_test = czech[0:train_size], czech[train_size:]
eston_train, eston_test = eston[0:train_size], eston[train_size:]
franc_train, franc_test = franc[0:train_size], franc[train_size:]
malay_train, malay_test = malay[0:train_size], malay[train_size:]
belgi_train, belgi_test = belgi[0:train_size], belgi[train_size:]
chile_train, chile_test = chile[0:train_size], chile[train_size:]
irela_train, irela_test = irela[0:train_size], irela[train_size:]
israe_train, israe_test = israe[0:train_size], israe[train_size:]
italy_train, italy_test = italy[0:train_size], italy[train_size:]
us_train, us_test = us[0:train_size], us[train_size:]
czech_train

,Unnamed: 0,date,new_deaths
0,62402,2020-03-01,NaN
1,62409,2020-03-08,0.0
2,62416,2020-03-15,0.0
3,62423,2020-03-22,0.0
4,62430,2020-03-29,17.0
...,...,...,...
156,63494,2023-02-26,13.0
157,63501,2023-03-05,-5.0
158,63508,2023-03-12,-7.0
159,63515,2023-03-19,21.0


## Model and Tuning